# PaleoSTeHM user interface

## Spatiotemporal model of Holocene sea-level change with Gaussian Process (no physical model)

We will use a Gaussian Process model with a global temporal kernel, a regionally linear ST kernel, a regional non-linear ST kernel, a locally non-linear ST kernel and a whitenoise kernel. We can write it down as:

$$
p(f| \Theta_{s},f ) \sim GP(0,k_{combined}(X,X')) \\
k_{combined}(X,X') = k_{global}(t,t') + k_{regionL}(X,X')  + k_{regionNL}(X,X') + k_{localNL}(X,X') + k_{whitenoise}(X,X')
$$

where $X$ indicates time and location of each sea-level data, consisting time ($t$) and location ($x$). Each specific kernel can be expressed as:

$$
k_{global}(t,t') = k_{M_{32}}(t,t')\\
k_{regionL}(X,X') = k_{Linear}(t,t') \cdot k_{RBF}(x,x')\\
k_{regionNL}(X,X') = k_{M_{32}}(t,t') \cdot k_{M_{32}}(x,x')\\
k_{localNL}(X,X') = k_{M_{32}}(t,t') \cdot k_{M_{32}}(x,x')\\
k_{whitenoise}(X,X') = k_{whitenose}(X,X')\\
$$

where $\cdot$ indicates element-wise multiplication. The difference between regional and local non-linear kernel is specified by different choices in kernels and prior distribtuions.

Here's a notebook for automatic implementation, optimization and visulization of RSL change, codes details are in `Holocene_SP_analysis.py`.


In [ ]:
# This is a workaround to avoid the torch error
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [ ]:
#load modules
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import sys
from pathlib import Path

sys.path.append('../..')
import PSTHM 
import Holocene_SP_analysis 
from PSTHM.config.load import load_config

#set plotting style
%matplotlib inline
font = {'weight':'normal',
       'size':20}

matplotlib.rc('font',**font)
matplotlib.rcParams['figure.figsize'] = (12, 6)
import warnings
warnings.filterwarnings("ignore")

config_path = Path("../../configs/holocene_stgp_UI.yaml").resolve()
config = load_config(str(config_path)).config

temporal_test_age = Holocene_SP_analysis.build_test_age_from_config(
    config, "temporal_test_age"
)
spatial_test_age = Holocene_SP_analysis.build_test_age_from_config(
    config, "spatial_test_age"
)

### Step 1 - loading your sea-level database

Please check `Data/US_Atlantic_Coast_for_ESTGP.csv` for data format.

You can replace the data to your own under the same format.


In [ ]:
outputs = Holocene_SP_analysis.load_rsl_data(config["paths"]["dataset"])
(
    rsl_age,
    rsl,
    rsl_sigma,
    rsl_1sigma_stack,
    rsl_age_sigma,
    rsl_age_1sigma_stack,
    ml_index,
    tl_index,
    slip_index,
    X_all,
    rsl_region,
    rsl_region_name,
    y,
    rsl_lat,
    rsl_lon,
) = outputs
ax = plt.subplot(111)
PSTHM.plotting.plot_uncertainty_boxes(
    rsl_age[slip_index],
    rsl[slip_index],
    rsl_age_1sigma_stack[slip_index] * 2,
    rsl_1sigma_stack[slip_index] * 2,
    ax=ax,
)
PSTHM.plotting.plot_limiting_data(
    rsl_age[ml_index],
    rsl[ml_index],
    rsl_age_1sigma_stack[ml_index] * 2,
    rsl_1sigma_stack[ml_index] * 2,
    marine_limiting=True,
    ax=ax,
)
PSTHM.plotting.plot_limiting_data(
    rsl_age[tl_index],
    rsl[tl_index],
    rsl_age_1sigma_stack[tl_index] * 2,
    rsl_sigma[tl_index] * 2,
    marine_limiting=False,
    ax=ax,
)
plt.xlim(12500, -50)
plt.ylim(-60, 20)
plt.legend(loc=0)

### Step 2 - loading your sea-level database

Implement and optimize spatiotemporal hierarchical model.

Config options in `configs/holocene_stgp_UI.yaml` under `gp_model`:

- `run_optimization: true` (set `false` to skip optimization)
- `fixed_seed: false` (set `true` to enable deterministic init)
- `seed: 42` (used when `fixed_seed: true`)
- `equal_kernels: null` (placeholder hyper-parameter)


In [ ]:
gpr = Holocene_SP_analysis.implement_sp_gp_model(
    X_all,
    y,
    rsl_sigma[slip_index],
    rsl_age_sigma[slip_index],
    initial_params=None,
    config=str(config_path),
)

In [ ]:
# # Advanced usage: use pre-defined hyper-parameters & skip optimization; use only when you know what you are doing
# gpr = Holocene_SP_analysis.implement_sp_gp_model(
#     X_all,
#     y,
#     rsl_sigma[slip_index],
#     rsl_age_sigma[slip_index],
#     initial_params=config.get("initial_params"),
#     run_optimization=False,
#     config=str(config_path),
# )

### Step 3 - visulizing modeled temporal rsl prediction for each region and saving the results


In [ ]:
Holocene_SP_analysis.process_rsl_data(
    rsl_region,
    rsl_region_name,
    rsl_lat,
    rsl_lon,
    rsl_age,
    rsl,
    rsl_age_1sigma_stack,
    rsl_1sigma_stack,
    rsl_sigma,
    slip_index,
    tl_index,
    ml_index,
    gpr,
    test_age=temporal_test_age,
)

### Step 4 - visulizing modeled spatial rsl prediction for each time and saving the results


In [ ]:
Holocene_SP_analysis.process_rsl_predictions_and_save(
    rsl_lat, rsl_lon, gpr, test_age=spatial_test_age
)